**NEURAL NETWORKS**

GRUP 4: Arnau Alabau, Sergi Ubach, María Casals


In [1]:
# Import matplotlib, numpy and math
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
#Codi per treure la sigmoide

def sigmoide(aux):
    x = np.linspace(-100, 100, 200)
    z = aux/(1 + np.exp(-(aux - 0)))

    #plt.plot(x, z)
    #plt.xlabel("x")
    #plt.ylabel("Sigmoid(X)")

    #plt.show()
    print("Z")
    print(z)
    return z

### Class Neuron

Aquesta classe preten definir una neurona de la xarxa neuronal

*Atributs:*
- nInputs: numero d'entrades que te la neurona
- output: valor de sortida
- bias: "offset" necessari
- weight: array ambs els pesos que determinen cada entrada


*Funcions:*
- calcula: ha de fer el calcul (inputs*weight + bias ) *sigmoide 
    - DUBTE: No se com es la operacio en concret


In [3]:
class Neuron:
    #Atributs
    nInputs = 0
    output = 0
    #nOutput? o sempre es un output?
    bias = 0
    weight = []
    
    def __init__(self, nInputs):
            self.nInputs = nInputs
            #generar weight i bias amb genetics
            self.weight = []
            self.bias = 0
            
            #per fer proves
            self.weight = np.array([0.2, 0.1, 0.3, 0.4, 0])
            
    def calcula(self, inputs):
        aux = inputs@self.weight + self.bias
        output = sigmoide(aux)
        
        return output

### Class Layer

Aquesta classe preten definir una capa de la xarxa neuronal

*Atributs:*
- nNeurons: numero de neurones que te la capa, coincideix amb el numero de outputs
- nInputs: numero d'entrades que te la capa


*Funcions:*
- calcula: ha de recorrer cada una de les neurones, passarli el vector d'entrada i obtenir la sortida de cada una


In [4]:
class Layer:
    #Atributs
    nNeurons = 5
    nInputs = 0
    neurons = [Neuron(3)]
    #nOutputs = 0
    #el num de neurones ve determinat pel numero de sortides, entenc que nOuputs = nNeurons
    
    def __init__(self, nNeurons, nInputs):
        self.nInputs = nInputs
        self.nNeurons = nNeurons
        #DUBTE: com s'inicialitzen les neurones?
        #self.neurons = [Neuron(3)]
    
    def calcula(self, inputs):
        for i in range (self.nNeurons):
            print(self.neurons[i].calcula(inputs))

### Class NeuralNetwork

Aquesta classe preten definir la xarxa neuronal sencera sobre la que treballarem

*Atributs:*
- nLayers: numero de capes que te la xarxa
- nInputs: numero d'entrades que te la xarxa
- nOuputs: numero de sortides que te la xarxa


*Funcions:*
- calcula: no se que fa jaja


In [5]:
class NeuralNetwork:
    #Atributs
    #DUBTE: no se quins atributs ha de tenir en concret
    nLayers = 0
    nInputs = 0
    nOutputs = 1
    
    def __init__(self, nLayers, nInputs, nOuputs): 
        self.nLayers = nLayers
        self.nInputs = nInputs
        self.nOuputs = nOuputs

        
    def calcula(self, inputs):
        print("calcula nn")
        #No tinc massa clar que ha de fer aquesta funcio

In [6]:
neuron = Neuron(5)
inputs = np.array([0, 1, 0, 0, 0])
neuron.calcula(inputs)

#La sortida d'una layer es l'entrada de la seguent
layer = Layer(1, 1)
layer.calcula(inputs)

Z
0.052497918747894
Z
0.052497918747894
0.052497918747894
